#### intro to gradient descent
fully connected neural networks are a sequence of matrices that map input vectors to output vectors by multiplying the matrices (layers) by intermediate vectors (activations). fitting these matrix products requires that we adjust the weights, or values of the matrices, so that when the input vectors pass through the matrices, we get predicted output vectors close to the true output vectors. 

we start with random weight values and use an optimization algoritm to get close to target output vector *y*. we initialize the matrix weights either manually (such as Xavier: var(weights) = 2/n<sub>in</sub> + n<sub>out</sub>), or with a deep learning library. the optimization algorithm first discussed in the fast.ai deep learning course is gradient descent. there are two flavors of gradient descent: standard and stochastic.

In [ ]:
%matplotlib inline

In [ ]:
import math, sys, os, numpy as np
from numpy.random import random
from matplotlib import pyplot as plt, rcParams, animation, rc
from ipywidgets import interact, interactive, fixed
from ipywidgets.widgets import *
rc('animation', html = 'html5')
rcParams['figure.figsize'] = 3, 3
%precision 4
np.set_printoptions(precision = 4, linewidth = 100)

stochastic gradient descent iteratively selects parameters (weights) to reduce the loss function, that is, the method of calculating the difference between the predicted outputs and the true outputs. to illustrate, we start with sum of squared errors as the loss function.  

In [ ]:
# start with a line of unknown parameters
def lin(a, b, x): return a*x+b

In [ ]:
# create parameters to start
a = 3
b = 8

In [ ]:
# randomly generated weights as matrix elements
# a powerful concept: starting with completely random weights yet finding a solution through iteration
n = 30
x = random(n)
y = lin(a, b, x)

In [ ]:
x

In [ ]:
y

In [ ]:
plt.scatter(x, y)

In [ ]:
# define our loss function: sum of squared errors
def sse(y, y_pred): return ((y-y_pred)**2).sum()
def loss(y, a, b, x): return sse(y, lin(a, b, x))
def avg_loss(y, a, b, x): return np.sqrt(loss(y, a, b, x)/n)

we want to minimize this averaged loss function value. our linear regression function has two 
numerical inputs: the *x* elements that represents the matrix values, and the *a*, *b* parameters we
specified above. we cannot change the matrix values, so we want our gradient descent optimizer to 
choose parameters that minimize the loss function value. 

In [ ]:
# the average loss is high if our guesses are bad, low if our guesses are good
a_guess = -1.
b_guess = 1.
avg_loss(y, a_guess, b_guess, x)

gradients are vectors that represent how the loss function changes with respect to each parameter. 
gradient descent entails iteratively calculating the partial derivative of the loss function with respect to each of our parameters and updating the parameters in the direction opposite that derivative. if the derivative of *a* is positive, increasing *a* increases the loss function. we get a positive gradient, so we want to move in the opposite direction by decreasing *a*. likewise, if the derivative is negative, the loss function decreases with each increase in *a* so we want to continue increasing *a*. in both cases, we want to move in the direction opposite the derivative.

gradient descent updates parameter values until it reaches parameter values that can no longer decrease the loss function value. this is the local minimum. 

the size of each of the algorithm's updates is the learning rate. high learning rates cover more ground more quickly, but have a higher risk of overshooting the local minimum. with lower learning rates, the algorithm more frequently takes steps (looks for negative gradients), thus increasing precision but at the expense of speed. 

In [ ]:
# define our learning rate as a little step 
# generally want the highest number we can get away with
lr = 0.01 

In [ ]:
# define our function for finding the optimal local minimum
# del[(y-(a*x+b))**2, b] = 2(b +a*x -y) = 2(y_pred - y)
# del[(y-(a*x+b))**2,a] = 2 x (b + a x - y)    = x * dy/db 
def update_weights(a, b, x, y, lr):
    global a_guess, b_guess
    y_pred = lin(a_guess, b_guess, x)
    dydb = 2 * (y_pred - y) # as b increases one unit, sse changes by 2 * (y_pred - y)
    dyda = x * dydb # as a increases one unit, sse changes by x * dydb
    a_guess -= lr * dyda.mean()
    b_guess -= lr * dydb.mean()

In [ ]:
# animate the change in sse loss function
fig = plt.figure(dpi = 100, figsize = (5, 4))
plt.scatter(x, y)
line, = plt.plot(x, lin(a_guess, b_guess, x))
plt.close()

def animate(i):
    line.set_ydata(lin(a_guess, b_guess, x))
    for i in range(10): 
        update_weights()
    return line

ani = animation.FuncAnimation(fig, animate, np.arange(0, 40), interval = 100)

In [ ]:
ani